<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/agent_ql_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
!pip install -q agentql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


### Install the Playwright dependency required by AgentQL.

In [ ]:
!playwright install chromium

164.5 MiB [] 0% 10.9s164.5 MiB [] 0% 36.0s164.5 MiB [] 0% 25.6s164.5 MiB [] 0% 17.0s164.5 MiB [] 0% 12.8s164.5 MiB [] 0% 10.2s164.5 MiB [] 1% 8.3s164.5 MiB [] 1% 7.3s164.5 MiB [] 1% 6.9s164.5 MiB [] 2% 6.2s164.5 MiB [] 2% 6.3s164.5 MiB [] 2% 6.1s164.5 MiB [] 3% 6.0s164.5 MiB [] 3% 5.9s164.5 MiB [] 3% 6.3s164.5 MiB [] 4% 6.2s164.5 MiB [] 4% 6.5s164.5 MiB [] 4% 6.6s164.5 MiB [] 4% 6.1s164.5 MiB [] 5% 6.0s164.5 MiB [] 5% 5.7s164.5 MiB [] 5% 5.8s164.5 MiB [] 6% 5.5s164.5 MiB [] 6% 5.4s164.5 MiB [] 7% 5.1s164.5 MiB [] 7% 4.9s164.5 MiB [] 8% 4.9s164.5 MiB [] 9% 5.0s164.5 MiB [] 9% 4.8s164.5 MiB [] 10% 4.8s164.5 MiB [] 10% 4.7s164.5 MiB [] 11% 4.7s164.5 MiB [] 11% 4.6s164.5 MiB [] 12% 4.6s164.5 MiB [] 13% 4.5s164.5 MiB [] 14% 4.5s164.5 MiB [] 14% 4.4s164.5 MiB [] 15% 4.4s164.5 MiB [] 15% 4.3s164.5 MiB [] 16% 4.2s164.5 MiB [] 17% 4.1s164.5 MiB [] 17% 4.2s164.5 MiB [] 18% 4.0s164.5 MiB [] 18% 3.9s164.5 MiB [] 19% 3.9s164.5 MiB [] 19% 3.8s164.5 MiB [] 20% 3.8s164.5 MiB [] 21% 3.7s164.5 MiB [] 22

You can [store](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) keys in Google Colab's Secrets.

In [ ]:
import os
from google.colab import userdata

os.environ["AGENTQL_API_KEY"]= userdata.get('AGENTQL_API_KEY')

### Extract Prices from 2 e-Commerce Sites

In [ ]:
import agentql
from playwright.async_api import async_playwright

url_1 = "https://www.amazon.ae/Apple-iPhone-15-256-GB/dp/B0CHXLB3WN/"
url_2 = "https://www.jumbo.ae/apple-iphone-15-5g-smartphone-blue-256-gb.html"


PRODUCT_INFO_QUERY = """
{
    price
}
"""

Run AgentQL script. Please note that online environment like Google Colab only supports **asynchronous version** of AgentQL.

In [ ]:
async with async_playwright() as playwright, await playwright.chromium.launch() as browser:
    # Create a new page in the browser and wrap it to get access to the AgentQL's querying API
    page = await agentql.wrap_async(browser.new_page())

    await page.goto(url_1)

    response = await page.query_data(PRODUCT_INFO_QUERY)

    print("Price at Amazon: ", response["price"])

    await page.goto(url_2)

    response = await page.query_data(PRODUCT_INFO_QUERY)

    print("Price at Jumbo: ", response["price"])

### Extract Prices by Searching

In [ ]:
import asyncio
import agentql
from playwright.async_api import BrowserContext, async_playwright

AMAZON_URL = "https://www.amazon.ae"
JUMBO_URL = "https://www.jumbo.ae/"

# Define the queries to interact with the page
HOME_PAGE_QUERY = """
{
    search_input
    search_button
}
"""

PRODUCT_INFO_QUERY = """
{
    product_price (for iPhone 15 256 GB)
}
"""

async def fetch_price(context: BrowserContext, session_url):
    """Open the given URL in a new tab and fetch the price of the product."""
    # Create a page in a new tab in the broswer context and wrap it to get access to the AgentQL's querying API
    page = await agentql.wrap_async(context.new_page())
    await page.goto(session_url)

    # Search for the product
    await page.wait_for_page_ready_state()
    home_response = await page.query_elements(HOME_PAGE_QUERY)
    await home_response.search_input.fill("iPhone 15 256 GB")
    await home_response.search_button.click()

    # Fetch the price data from the page
    data = await page.query_data(PRODUCT_INFO_QUERY)
    return data["product_price"]


async with async_playwright() as playwright, await playwright.chromium.launch(
    ) as browser, await browser.new_context() as context:
        # Open multiple tabs in the same browser context to fetch prices concurrently
        (amazon_price, jumbo_price) = await asyncio.gather(
            fetch_price(context, AMAZON_URL),
            fetch_price(context, JUMBO_URL),
        )

        print(
            f"""
        Price at Amazon : {amazon_price}
        Price at Jumbo : {jumbo_price}
        """
        )

In [ ]:
import asyncio
import agentql
from playwright.async_api import BrowserContext, async_playwright

AMAZON_URL = "https://www.amazon.ae"
JUMBO_URL = "https://www.jumbo.ae/"

# Define the queries to interact with the page
HOME_PAGE_QUERY = """
{
    search_input
    search_button
}
"""

PRODUCT_INFO_QUERY = """
{
    product_price (for iPhone 15 256 GB)
}
"""

### Get By Prompt

In [ ]:
async def fetch_price(context: BrowserContext, session_url):
    """Open the given URL in a new tab and fetch the price of the product."""
    # Create a page in a new tab in the broswer context and wrap it to get access to the AgentQL's querying API
    page = await agentql.wrap_async(context.new_page())
    await page.goto(session_url)

    # Search for the product
    await page.wait_for_page_ready_state()
    search_text_box = await page.get_by_prompt(prompt="Search input field")
    search_button_box = await page.get_by_prompt(prompt="Search button")
    await search_text_box.fill("iPhone 15 256 GB")
    await search_button_box.click()

    # Fetch the price data from the page
    data = await page.query_data(PRODUCT_INFO_QUERY)
    return data["product_price"]


async with async_playwright() as playwright, await playwright.chromium.launch(
    ) as browser, await browser.new_context() as context:
        # Open multiple tabs in the same browser context to fetch prices concurrently
        amazon_price = await asyncio.gather(
            fetch_price(context, AMAZON_URL),
        )

        print(
            f"""
        Price at Amazon : {amazon_price}
        """
        )


        Price at Amazon : ['AED 3,299.00']
        
